# Solving a Sudoku with AI
This notebook was made for solving the quizes and exercies in the AI nanodegree from Udacity.

## Coding the board

In [51]:
rows = 'ABCDEFGHI'
cols = '123456789'
possible_digits = '123456789'

def cross(a, b):
    """
    Return a list with all concatenations of a letter in
    `a` with a letter in `b`
    
    Args:
        a: A string
        b: A string
    Returns:
        A list formed by all the possible concatenations of a
        letter in `a` with a letter in `b`
    """
    return [s + t for s in a for t in b]

boxes = cross(rows, cols)
print(boxes)

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9']


In [52]:
# Lets get all the row units
# row_units[0] = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
row_units = [cross(row, cols) for row in rows]

# Same for column units
# column_units[0] = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
column_units = [cross(rows, col) for col in cols]

# And now for square units
# square_units[0] = ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']
square_units = [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')]

unitlist = row_units + column_units + square_units
units = dict((box, [unit for unit in unitlist if box in unit]) for box in boxes)
peers = dict((box, set(sum(units[box], []))-set([box])) for box in boxes)

print('First row unit: ', row_units[0])
print('First col unit: ', column_units[0])
print('First sqr unit: ', square_units[0])

First row unit:  ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9']
First col unit:  ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1']
First sqr unit:  ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']


In [53]:
def display(values):
    """
    Prints the values of a sudoku as a 2-D grid.
    
    Args:
        values: A dict representing a sodoku.
        
    Returns:
        `None`
    """
    width = 1 + max(len(values[s]) for s in boxes)
    line = '+'.join(['-' * (width * 3)] * 3)
    for row in rows:
        print(''.join(values[row + col].center(width) + ('|' if col in '36' else '') for col in cols))
        if row in 'CF': print(line)
    return

def grid_values(grid):
    """
    Returns a dict that represents a sudoku.
    
    Args:
        grid: A string with the starting numbers
        for all the boxes in a sudoku. Empty boxes
        can be represented as dots `.`.
        Example: `'..3.2.6.'...`
        
    Returns:
        A dict that represents a sudoku. The keys
        will be the boxes labels and it's value will be the number
        or a dot `.` if the box is empty.
    """
    assert len(grid) == 81, "The lenght of `grid` should be 81. A 9x9 sudoku"
    chars = []
    for char in grid:
        if char in possible_digits: chars.append(char)
        if char == '.': chars.append(possible_digits)
    return dict(zip(boxes, chars))



sudoku_starting_dict = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
display(sudoku_starting_dict)


123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3   

## Strategy 1: Elimination
If a box has a value assigned, the none of the peers of this box can have this value
![Image from udacity AI nanodegree](https://i.imgur.com/GID0eWu.png)

In [54]:
def eliminate(sudoku_dict):
    """
    Returns a sudoku dict after applying the eliminate technique.
    
    Args:
        sudoku_dict: A dict representing the sudoku. It'll contain
        in each box the value of it, or the possible values.
        
    Returns:
        A sudoku dict after applying the eliminate technique in all
        the boxes.
    """
    solved_values = [box for box in sudoku_dict.keys() if len(sudoku_dict[box]) == 1]
    for box in solved_values:
        value = sudoku_dict[box]
        for peer in peers[box]:
            sudoku_dict[peer] = sudoku_dict[peer].replace(value, '')
            
    return sudoku_dict

sudoku_after_eliminate = eliminate(sudoku_starting_dict)
print('\n Sudoku after eliminate technique. \n')
display(sudoku_after_eliminate)


 Sudoku after eliminate technique. 

   45    4578    3   |   49     2     147  |   6     5789    57  
   9    24678    47  |   3      47     5   |   78    278     1   
   25    257     1   |   8      79     6   |   4    23579   2357 
---------------------+---------------------+---------------------
  345    345     8   |   1     3456    2   |   9    34567  34567 
   7    123459   49  |  459   34569    4   |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  134    1347    2   |   6     478     9   |   5     1478    47  
   8     1467    47  |   2     457     3   |   17    1467    9   
   46    4679    5   |   4      1      47  |   3    24678   2467 


## Strategy 2: Only Choice
If there is a box in a unit which would allow a certain digit, then that bux must be assigned that digit
![Image from udacity AI nanodegree](https://i.imgur.com/G3ACj8v.png)

In [55]:
def only_choice(sudoku_dict):
    """
    It runs through all the units of a sudoku
    and it applies the only choice technique.
    
    Args: 
        sudoku_dict: A dict representing the sudoku.
    Returns:
        A sudoku dict after applying the only choice technique.
    """
    for unit in unitlist:
        for value in possible_digits:
            value_places = [box for box in unit if value in sudoku_dict[box]]
            if len(value_places) == 1:
                sudoku_dict[value_places[0]] = value
                
    return sudoku_dict

sudoku_after_only_choice= only_choice(sudoku_after_eliminate)
display(sudoku_after_only_choice)

  45    8     3   |  9     2     1   |  6    5789   57  
  9     6     47  |  3     4     5   |  8    278    1   
  2    257    1   |  8     7     6   |  4   23579  2357 
------------------+------------------+------------------
 345   345    8   |  1    3456   2   |  9   34567 34567 
  7     2     9   |  5   34569   4   |  1   13456   8   
 1345 13459   6   |  7    3459   8   |  2    1345  345  
------------------+------------------+------------------
 134   1347   2   |  6     8     9   |  5    1478   47  
  8    1467   47  |  2     5     3   |  17    6     9   
  6     9     5   |  4     1     7   |  3     8     2   
